# Análise dos resultados parciais da extração

## Pré-processamento

Tratando datas e tipos de dados

In [108]:
import pandas as pd
import matplotlib as plt
import plotly

In [109]:
repo = pd.read_csv("../data/repositories_1573218408.csv",  dtype={'id': object,
                                                                'timestamp_extract': object,
                                                                'commits': pd.Int64Dtype(),
                                                                'contributors': pd.Int64Dtype()})

In [110]:
repo.dtypes

id                   object
full_name            object
description          object
owner_type           object
owner_api_url        object
owner_url            object
url                  object
api_url              object
fork                   bool
created_at           object
updated_at           object
size                  int64
stargazers_count      int64
language             object
has_issues             bool
has_wiki               bool
forks_count           int64
forks                 int64
open_issues           int64
watchers              int64
timestamp_extract    object
commits               Int64
contributors          Int64
dtype: object

In [130]:
repo.describe()

,size,stargazers_count,forks_count,forks,open_issues,watchers,commits,contributors,created_at_mes_numero,created_at_ano,created_at_dia,updated_at_mes_numero,updated_at_ano,updated_at_dia
count,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,549.000000,549.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000
mean,8407.038596,3.307018,0.992982,0.992982,0.857895,3.307018,61.810565,1.954463,6.882456,2016.889474,15.438596,7.417544,2017.612281,15.696491
std,37510.085907,13.094578,3.828478,3.828478,8.732048,13.094578,808.848007,13.424557,3.265930,1.920217,8.507451,3.243657,1.656491,9.060090
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2010.000000,1.000000,1.000000,2013.000000,1.000000
25%,30.500000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,1.000000,4.000000,2016.000000,8.000000,5.000000,2017.000000,7.000000
50%,443.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,7.000000,2017.000000,16.000000,8.000000,2018.000000,16.000000
75%,3701.250000,1.000000,1.000000,1.000000,0.000000,1.000000,25.000000,1.000000,9.000000,2018.000000,22.000000,10.000000,2019.000000,24.000000
max,555527.000000,135.000000,43.000000,43.000000,203.000000,135.000000,18911.000000,315.000000,12.000000,2019.000000,31.000000,12.000000,2019.000000,31.000000


In [111]:
repo.head(3)[['id', 'url', 'created_at', 'updated_at', 'timestamp_extract']]

,id,url,created_at,updated_at,timestamp_extract
0,29256552,https://github.com/CamaraDosDeputados/dados-ab...,2015-01-14T17:32:49Z,2019-11-05T21:58:29Z,1573215945
1,39256926,https://github.com/dadosgovbr/catalogos-dados-...,2015-07-17T14:02:34Z,2019-10-25T17:38:20Z,1573215945
2,73385196,https://github.com/prefeiturasp/dados-educacao,2016-11-10T13:35:40Z,2019-08-17T00:49:49Z,1573215945


In [112]:
def conversao_string_datetime(df, coluna):
    df[coluna] = pd.to_datetime(df[coluna], dayfirst=True)
    df[coluna + '_mes'] = df[coluna].dt.month_name(locale='pt_BR.UTF-8')
    df[coluna + '_mes_numero'] = df[coluna].dt.month
    df[coluna + '_ano'] = df[coluna].dt.year
    df[coluna + '_dia'] = df[coluna].dt.day

    return df

In [113]:
repo = conversao_string_datetime(repo, 'created_at')
repo = conversao_string_datetime(repo, 'updated_at')

repo.head(3)

,id,full_name,description,owner_type,owner_api_url,owner_url,url,api_url,fork,created_at,...,commits,contributors,created_at_mes,created_at_mes_numero,created_at_ano,created_at_dia,updated_at_mes,updated_at_mes_numero,updated_at_ano,updated_at_dia
0,29256552,CamaraDosDeputados/dados-abertos,Repositório do serviço de Dados Abertos da Câm...,Organization,https://api.github.com/users/CamaraDosDeputados,https://github.com/CamaraDosDeputados,https://github.com/CamaraDosDeputados/dados-ab...,https://api.github.com/repos/CamaraDosDeputado...,False,2015-01-14 17:32:49+00:00,...,28,4,Janeiro,1,2015,14,Novembro,11,2019,5
1,39256926,dadosgovbr/catalogos-dados-brasil,Mapeamento de iniciativas (e catálogos) de dad...,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/catalogos-dados-...,https://api.github.com/repos/dadosgovbr/catalo...,False,2015-07-17 14:02:34+00:00,...,42,4,Julho,7,2015,17,Outubro,10,2019,25
2,73385196,prefeiturasp/dados-educacao,Análises e tutoriais das bases de dados aberto...,Organization,https://api.github.com/users/prefeiturasp,https://github.com/prefeiturasp,https://github.com/prefeiturasp/dados-educacao,https://api.github.com/repos/prefeiturasp/dado...,False,2016-11-10 13:35:40+00:00,...,18,2,Novembro,11,2016,10,Agosto,8,2019,17


In [120]:
id_repos = repo.loc[repo['commits'].isna()]['id']
len(id_repos)

21

### Verificando quais são os repositórios com commits e contributors nulos

In [131]:
def listar_registros_nulos(df, campoNulo, campoChave ):
    
    listaCampochave = repo.loc[repo[campoNulo].isna()][campoChave]
    return df.loc[repo[campoChave].isin(listaCampochave)]

In [132]:
listar_registros_nulos(repo, 'commits', 'id')

,id,full_name,description,owner_type,owner_api_url,owner_url,url,api_url,fork,created_at,...,commits,contributors,created_at_mes,created_at_mes_numero,created_at_ano,created_at_dia,updated_at_mes,updated_at_mes_numero,updated_at_ano,updated_at_dia
146,193599332,renatachagasc/Api-DadosAbertos-Eventos,NaN,User,https://api.github.com/users/renatachagasc,https://github.com/renatachagasc,https://github.com/renatachagasc/Api-DadosAber...,https://api.github.com/repos/renatachagasc/Api...,False,2019-06-25 00:03:30+00:00,...,NaN,NaN,Junho,6,2019,25,Junho,6,2019,25
152,13354150,ccdpoa/ocida,Observatórios Climáticos Independentes de Dado...,User,https://api.github.com/users/ccdpoa,https://github.com/ccdpoa,https://github.com/ccdpoa/ocida,https://api.github.com/repos/ccdpoa/ocida,False,2013-10-05 23:10:37+00:00,...,NaN,NaN,Outubro,10,2013,5,Outubro,10,2013,5
159,176943542,Kassio-Ferreira/perfil_racial_ufersa,Exemplo de aplicação dos dados abertos.,User,https://api.github.com/users/Kassio-Ferreira,https://github.com/Kassio-Ferreira,https://github.com/Kassio-Ferreira/perfil_raci...,https://api.github.com/repos/Kassio-Ferreira/p...,False,2019-03-21 12:49:59+00:00,...,NaN,NaN,Março,3,2019,21,Março,3,2019,21
173,212679438,GabrielLimaSnT/ProjetoOpeDadosAbertos,Projeto de Dados Abertos do Governo Ope ~Turma...,User,https://api.github.com/users/GabrielLimaSnT,https://github.com/GabrielLimaSnT,https://github.com/GabrielLimaSnT/ProjetoOpeDa...,https://api.github.com/repos/GabrielLimaSnT/Pr...,False,2019-10-03 21:00:56+00:00,...,NaN,NaN,Outubro,10,2019,3,Outubro,10,2019,3
203,218686494,hugomsouto/mineracao-dados-abertos,NaN,User,https://api.github.com/users/hugomsouto,https://github.com/hugomsouto,https://github.com/hugomsouto/mineracao-dados-...,https://api.github.com/repos/hugomsouto/minera...,False,2019-10-31 04:49:29+00:00,...,NaN,NaN,Outubro,10,2019,31,Outubro,10,2019,31
211,209605170,DINALVAGOMES/INSS_dados-abertos,NaN,User,https://api.github.com/users/DINALVAGOMES,https://github.com/DINALVAGOMES,https://github.com/DINALVAGOMES/INSS_dados-abe...,https://api.github.com/repos/DINALVAGOMES/INSS...,False,2019-09-19 16:53:05+00:00,...,NaN,NaN,Setembro,9,2019,19,Setembro,9,2019,19
213,144884415,eduponto21/dados_abertos_curitiba_disque_economia,"Dados abertos de Curitiba-PR, referentes ao Di...",User,https://api.github.com/users/eduponto21,https://github.com/eduponto21,https://github.com/eduponto21/dados_abertos_cu...,https://api.github.com/repos/eduponto21/dados_...,False,2018-08-15 17:42:03+00:00,...,NaN,NaN,Agosto,8,2018,15,Agosto,8,2018,15
226,86007528,MarxSteel/MDIO-InteractBrasil,Página de Dados Abertos da MDIO Interact Brasil,User,https://api.github.com/users/MarxSteel,https://github.com/MarxSteel,https://github.com/MarxSteel/MDIO-InteractBrasil,https://api.github.com/repos/MarxSteel/MDIO-In...,False,2017-03-23 23:33:27+00:00,...,NaN,NaN,Março,3,2017,23,Março,3,2017,23
230,59862291,yelken/livecity,WebHook para varrer os dados abertos [Hackacit...,User,https://api.github.com/users/yelken,https://github.com/yelken,https://github.com/yelken/livecity,https://api.github.com/repos/yelken/livecity,False,2016-05-27 20:44:07+00:00,...,NaN,NaN,Maio,5,2016,27,Maio,5,2016,27
276,128829155,danielmbicalho/Dados_reposit-rios_e_ferramentas,Com este pequeno projeto pretendemos coletar r...,User,https://api.github.com/users/danielmbicalho,https://github.com/danielmbicalho,https://github.com/danielmbicalho/Dados_reposi...,https://api.github.com/repos/danielmbicalho/Da...,False,2018-04-09 20:21:31+00:00,...,NaN,NaN,Abril,4,2018,9,Abril,4,2018,9


In [129]:
listar_registros_nulos(repo, 'contributors', 'id')[['contributors', 'commits']]

,contributors,commits
146,NaN,NaN
152,NaN,NaN
159,NaN,NaN
173,NaN,NaN
203,NaN,NaN
211,NaN,NaN
213,NaN,NaN
226,NaN,NaN
230,NaN,NaN
276,NaN,NaN


## Agrupando dados

In [136]:
repo.columns

Index(['id', 'full_name', 'description', 'owner_type', 'owner_api_url',
       'owner_url', 'url', 'api_url', 'fork', 'created_at', 'updated_at',
       'size', 'stargazers_count', 'language', 'has_issues', 'has_wiki',
       'forks_count', 'forks', 'open_issues', 'watchers', 'timestamp_extract',
       'commits', 'contributors', 'created_at_mes', 'created_at_mes_numero',
       'created_at_ano', 'created_at_dia', 'updated_at_mes',
       'updated_at_mes_numero', 'updated_at_ano', 'updated_at_dia'],
      dtype='object')

### Tipos de usuários do github

In [140]:
repo.groupby('owner_type').size().reset_index(name='Quantidade')

,owner_type,Quantidade
0,Organization,104
1,User,466


### Linguagens de Programação

In [142]:
repo.groupby('language').size().reset_index(name='Quantidade').sort_values('Quantidade', ascending=False)

,language,Quantidade
14,Python,92
9,JavaScript,68
10,Jupyter Notebook,59
7,HTML,52
13,PHP,33
8,Java,29
15,R,20
4,CSS,16
16,Ruby,15
20,TypeScript,9
